In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisosONEHOT.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantesONEHOT.csv")
areas=avisos['nombre_area'].value_counts()
avisos['nombre_area']=avisos['nombre_area'].replace(areas.index, areas.values)

avisos_no_one_hot= pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
avisos_no_one_hot=avisos_no_one_hot[['idaviso','tipo_de_trabajo','nivel_laboral','nombre_area']];
avisos_no_one_hot['tipo_de_trabajo']=avisos_no_one_hot['tipo_de_trabajo'].astype('category')
avisos_no_one_hot['nivel_laboral']=avisos_no_one_hot['nivel_laboral'].astype('category')
avisos_no_one_hot['nombre_area']=avisos_no_one_hot['nombre_area'].astype('category')
avisos_no_one_hot=avisos_no_one_hot.rename(columns={'nombre_area': 'nombre_area_aux'})

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante
0,1111780242,YjVJQ6Z
1,1112263876,BmVpYoR
2,1112327963,wVkBzZd
3,1112318643,OqmP9pv
4,1111903673,DrpbXDP


In [5]:
vistas_por_area=vistas.copy()
vistas_por_area['vistas_por_area']=0;
vistas_por_area=pd.merge(vistas_por_area,avisos_no_one_hot,on='idaviso', how='left')
vistas_por_area=vistas_por_area.groupby(['idpostulante','nombre_area_aux'],as_index=False).agg({'vistas_por_area':'count'})
vistas_por_area.head()

,idpostulante,nombre_area_aux,vistas_por_area
0,0005E,Tecnologia / Sistemas,2
1,00Lkv,Comercio Exterior,2
2,00dMd,Comercio Exterior,4
3,01QAq,Contabilidad,1
4,021OM,Comercio Exterior,5


In [6]:
vistas_por_nivel=vistas.copy()
vistas_por_nivel['vistas_por_nivel']=0;
vistas_por_nivel=pd.merge(vistas_por_nivel,avisos_no_one_hot,on='idaviso', how='left')
vistas_por_nivel=vistas_por_nivel.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})
vistas_por_nivel.head()

,idpostulante,nivel_laboral,vistas_por_nivel
0,0005E,Otro,2
1,00Lkv,Otro,2
2,00dMd,Otro,4
3,01QAq,Otro,1
4,021OM,Otro,5


In [7]:
vistas_por_tipo=vistas.copy()
vistas_por_tipo['vistas_por_tipo']=0;
vistas_por_tipo=pd.merge(vistas_por_tipo,avisos_no_one_hot,on='idaviso', how='left')
vistas_por_tipo=vistas_por_tipo.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_por_tipo.head()

,idpostulante,tipo_de_trabajo,vistas_por_tipo
0,0005E,Full-time,2
1,00Lkv,Full-time,2
2,00dMd,Full-time,4
3,01QAq,Full-time,1
4,021OM,Full-time,5


In [8]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [9]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [10]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [11]:
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [12]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [13]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})

In [14]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/12.csv").sample(len(postulaciones))
print(len(postulaciones2))
postulaciones2

6604534


,idaviso,idpostulante,sepostulo
3870536,1112285457,dYJdJlP,0
2460605,1112460196,6Rbmvx,0
3366416,1112359653,KBaYLpd,0
3399245,1112211853,A3Xw2Nj,0
3286214,1112217481,rmd0O3Q,0
4443628,1002149954,YjVV3qX,0
3739074,1112396133,PmqkPD0,0
3817759,1112436420,zvxX1ZE,0
2904070,1112017530,aMj8lN,0
6397436,1112390604,zv858pD,0


In [15]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [16]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

13209068
13203714


In [17]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,avisos_no_one_hot,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad
2961114,1112278621,X95K3NJ,1,28.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Ventas,NaN,NaN,NaN
3452627,1112268282,A3XQleY,1,18.0,True,False,False,False,False,False,...,False,True,False,False,Part-time,Junior,Call Center,NaN,NaN,NaN
1528459,1112191442,4reGl80,0,30.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Telecomunicaciones,7.0,NaN,NaN
2921776,1112285247,vVDm95j,1,31.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Ventas,3.0,NaN,NaN
4300406,1112267399,owEldLj,0,22.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Business Intelligence,36.0,NaN,NaN
2822003,1112248829,8MZpxvL,1,37.0,False,True,False,False,False,False,...,False,False,True,False,Part-time,Otro,Atención al Cliente,1.0,NaN,NaN
11905386,1112504616,aVkvqq,0,25.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Instrumentación quirúrgica,8.0,NaN,NaN
9797835,1111397360,ZDrzr55,0,33.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Ventas,13.0,NaN,NaN
5237349,1000641565,Yj59WpJ,0,200.0,False,False,True,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Ventas,6.0,NaN,NaN
2284122,1112441879,NDb80B,1,32.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Administración,78.0,1.0,0.012821


In [18]:
postulaciones['cantidad']=postulaciones['cantidad'].fillna(0)
postulaciones['vistas_postulante']=postulaciones['vistas_postulante'].fillna(0)
postulaciones['vistas_al_aviso']=postulaciones['vistas_al_aviso'].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad,lo_vio
3256631,1112289599,2zPDkEv,1,25.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Ventas,2.0,0.0,0.000000,False
8678315,1112354253,8MadkmM,1,22.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Finanzas,123.0,0.0,0.000000,False
8803066,1112444138,bOjYNO4,0,25.0,True,False,False,False,False,False,...,False,False,False,Full-time,Jefe / Supervisor / Responsable,Atención al Cliente,0.0,0.0,0.000000,False
5462455,1111904339,LNKJ2jB,0,22.0,False,True,False,False,False,False,...,True,False,False,Full-time,Junior,Transporte,13.0,0.0,0.000000,False
217583,1112308247,96MdEPv,1,28.0,False,True,False,False,False,False,...,False,False,False,Full-time,Jefe / Supervisor / Responsable,Administración,244.0,0.0,0.000000,False
5611281,1112469138,1KPoMB,1,36.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Tesorería,11.0,1.0,0.090909,True
1733483,1112398356,NO84b4,1,37.0,True,False,False,False,False,False,...,False,False,True,Part-time,Senior / Semi-Senior,Recepcionista,42.0,4.0,0.095238,True
22058,1112315170,NzMo6GV,1,30.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Contabilidad,85.0,0.0,0.000000,False
1296348,1112340363,3NPBJDa,0,18.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Ingeniería Electromecánica,9.0,0.0,0.000000,False
8218415,1112423782,pzdDp4L,0,19.0,True,False,False,False,False,False,...,True,False,False,Full-time,Junior,Contabilidad,1.0,0.0,0.000000,False


In [19]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area_aux'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area_aux,postulaciones_misma_area
0,0z5Dmrd,Compensación y Planilla,1
1,0z5Dmrd,Contabilidad,1
2,0z5Dmrd,Control de Gestión,1
3,0z5Dmrd,Diseño Industrial,1
4,0z5Dmrd,Farmacéutica,1


In [20]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,Otro,1
1,0z5Dmrd,Senior / Semi-Senior,15
2,0z5JW1r,-,1
3,0z5JW1r,Gerencia / Alta Gerencia / Dirección,2
4,0z5JW1r,Jefe / Supervisor / Responsable,3


In [21]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,16
1,0z5JW1r,Full-time,20
2,0z5JW1r,Part-time,2
3,0z5JW1r,Por Horas,1
4,0z5VvGv,Full-time,53


In [22]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones=pd.merge(postulaciones,vistas_por_area,on=['idpostulante','nombre_area_aux'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
6792084,1112425888,DrppDlX,1,31.0,True,False,False,False,False,False,...,109.0,6.0,0.055046,True,0,6,4,6.0,35.0,35.0
7696644,1112355344,Yj5dw4D,1,21.0,False,True,False,False,False,False,...,3.0,0.0,0.000000,False,11,24,18,3.0,3.0,3.0
268411,1112394221,jk6o3jQ,1,45.0,False,True,False,False,False,False,...,50.0,12.0,0.240000,True,10,23,15,43.0,42.0,42.0
9909391,1112461955,3NkAw2a,0,26.0,True,False,False,False,False,False,...,5.0,0.0,0.000000,False,0,60,10,0.0,5.0,5.0
12546310,1111447672,aV3ooJ,0,59.0,False,True,False,False,False,False,...,1.0,0.0,0.000000,False,0,15,6,0.0,1.0,1.0
8367305,1112454272,Pm4roMv,0,50.0,True,False,False,False,False,False,...,4.0,0.0,0.000000,False,1,10,6,0.0,4.0,4.0
2754543,1111796567,jkVbajQ,1,41.0,True,False,False,False,False,False,...,65.0,3.0,0.046154,True,2,19,21,8.0,28.0,28.0
3557828,1112423965,ak4E8rW,1,26.0,True,False,False,False,False,False,...,49.0,1.0,0.020408,True,1,33,8,1.0,35.0,35.0
5332889,1112208794,96zqQBY,1,29.0,False,True,False,False,False,False,...,102.0,1.0,0.009804,True,4,21,20,13.0,81.0,81.0
5068178,1112401940,6K94Zl,0,27.0,True,False,False,False,False,False,...,4.0,0.0,0.000000,False,0,12,1,0.0,4.0,4.0


In [23]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))

Train:  12148496 Test:  639395


In [24]:
features=['nombre_area', 'edad','sexo_FEM', 'sexo_MASC','Doctorado_Abandonado',
'Doctorado_En Curso', 'Doctorado_Graduado', 'Master_Abandonado',
'Master_En Curso', 'Master_Graduado', 'Posgrado_Abandonado',
'Posgrado_En Curso', 'Posgrado_Graduado', 'Secundario_Abandonado',
'Secundario_En Curso', 'Secundario_Graduado', 'Terciario/Técnico_Abandonado',
'Terciario/Técnico_En Curso', 'Terciario/Técnico_Graduado', 'Universitario_Abandonado',
'Universitario_En Curso', 'Universitario_Graduado', 'pide_hombre',
'pide_mujer', 'ingles', 'experiencia', 'paquete_office', 'liderazgo', 'secundario',
'viajar', 'empresa_importante', 'capacitacion', 'remuneracion_pretendida', 'graduados',
'estudiantes', 'conocimientos_especificos', 'buen_ambiente', 'discriminacion',
'tipo_de_trabajo_Fines de Semana', 'tipo_de_trabajo_Full-time', 'tipo_de_trabajo_Part-time',
'tipo_de_trabajo_Pasantia', 'tipo_de_trabajo_Por Contrato', 'tipo_de_trabajo_Por Horas',
'tipo_de_trabajo_Primer empleo', 'tipo_de_trabajo_Teletrabajo', 'tipo_de_trabajo_Temporario',
'tipo_de_trabajo_Voluntario', 'nivel_laboral_Gerencia / Alta Gerencia / Dirección',
'nivel_laboral_Jefe / Supervisor / Responsable', 'nivel_laboral_Junior',
'nivel_laboral_Otro', 'nivel_laboral_Senior / Semi-Senior', 'vistas_postulante',
'vistas_al_aviso', 'cantidad', 'postulaciones_misma_area', 'postulaciones_mismo_tipo', 
'postulaciones_mismo_nivel','lo_vio','vistas_por_area','vistas_por_nivel','vistas_por_tipo']

In [25]:
x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

In [26]:
postulaciones=0
c=0
gc.collect()

128

In [27]:
c=GaussianNB()
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")
dt=c.fit(x_train,y_train)

In [28]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=c.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.8728245112443435


In [29]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/Naive Bayes.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/Naive Bayes.pkl']

In [30]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos_no_one_hot,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion['lo_vio']=(prediccion['vistas_al_aviso']>0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion=pd.merge(prediccion,vistas_por_area,on=['idpostulante','nombre_area_aux'],how='left')
prediccion=pd.merge(prediccion,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.sample(10)

100000
100000
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
27373,52631,1112443859,NZblpx,26.0,True,False,False,False,False,False,...,180.0,2.0,0.011111,True,1.0,50.0,11.0,2.0,104.0,104.0
12092,80053,1112464787,YZrAoX,33.0,True,False,False,False,False,False,...,417.0,1.0,0.002398,True,27.0,112.0,10.0,30.0,305.0,305.0
54078,74008,1112463532,eOdeV9,36.0,False,True,False,False,False,False,...,2.0,0.0,0.000000,False,0.0,7.0,3.0,0.0,2.0,2.0
66046,55222,1112447428,N0Gr2L,44.0,False,True,False,False,False,False,...,11.0,0.0,0.000000,False,0.0,3.0,1.0,0.0,11.0,11.0
32492,12176,1111985420,YjAJZ0P,36.0,False,True,False,False,False,False,...,45.0,1.0,0.022222,True,5.0,13.0,9.0,10.0,36.0,36.0
51814,57213,1112451167,8wJwLL,67.0,False,True,False,False,False,False,...,2.0,0.0,0.000000,False,0.0,3.0,0.0,0.0,2.0,2.0
1677,86685,1112466769,EW4vmz,27.0,True,False,False,False,False,False,...,11.0,0.0,0.000000,False,0.0,0.0,0.0,0.0,6.0,6.0
26767,88670,1112467018,Dr6BobX,35.0,True,False,False,False,False,False,...,42.0,2.0,0.047619,True,2.0,6.0,4.0,2.0,26.0,26.0
79354,41000,1112420394,EAwB2,43.0,False,True,False,False,False,False,...,7.0,0.0,0.000000,False,0.0,4.0,2.0,0.0,5.0,5.0
72196,38319,1112414881,E46Qm9,51.0,False,True,False,False,False,False,...,1.0,0.0,0.000000,False,0.0,3.0,1.0,0.0,1.0,1.0


In [31]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[1.00000000e+00, 1.03216942e-10],
       [1.00000000e+00, 2.74955689e-12],
       [1.00000000e+00, 1.27401573e-12],
       ...,
       [9.99999225e-01, 7.75399083e-07],
       [9.99999951e-01, 4.87862821e-08],
       [9.99999977e-01, 2.30175707e-08]])

In [32]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [33]:
prediccion['sepostulo']=y_final2

In [34]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,0.0,0.0,False,0.0,2.0,0.0,4.0,24.0,24.0,1.032169e-10
1,1,739260,6v1xdL,30.0,False,True,False,False,False,False,...,0.0,0.0,False,5.0,44.0,0.0,0.0,6.0,6.0,2.749557e-12
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,0.0,0.0,False,0.0,1.0,0.0,1.0,7.0,7.0,1.274016e-12
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,5.229340e-08
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,0.0,0.0,False,0.0,1.0,0.0,0.0,3.0,3.0,3.073328e-08


In [35]:
prediccion=prediccion[['id','sepostulo']]
prediccion.head()

,id,sepostulo
0,0,1.032169e-10
1,1,2.749557e-12
2,2,1.274016e-12
3,3,5.229340e-08
4,4,3.073328e-08


In [37]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion54.csv",index=False)

In [36]:
prediccion.sepostulo.mean()

0.4039625612902181